In [ ]:
import torch

res = 60
xx, yy, zz = torch.meshgrid([
    torch.linspace(-0.5, 0.5, res),  
    torch.linspace(-0.5, 0.5, res),  
    torch.linspace(-0.5, 0.5, res),
], indexing='ij')
vertices = torch.stack([
    xx.flatten(), yy.flatten(), zz.flatten()
], dim=-1)

ind_grid = torch.arange(0, vertices.shape[0]).view(res, res, res)
s_0, s_1 = slice(None, -1), slice(1, None)
indices_grid = torch.stack([
    torch.stack([ind_grid[s_0,s_0,s_0], ind_grid[s_1,s_0,s_1], 
                 ind_grid[s_1,s_0,s_0], ind_grid[s_0,s_1,s_0]], dim=-1),
    torch.stack([ind_grid[s_1,s_0,s_0], ind_grid[s_1,s_1,s_0], 
                 ind_grid[s_0,s_1,s_0], ind_grid[s_1,s_0,s_1]], dim=-1),
    torch.stack([ind_grid[s_0,s_1,s_0], ind_grid[s_1,s_1,s_0], 
                 ind_grid[s_1,s_1,s_1], ind_grid[s_1,s_0,s_1]], dim=-1),
    torch.stack([ind_grid[s_0,s_1,s_0], ind_grid[s_0,s_1,s_1], 
                 ind_grid[s_1,s_0,s_1], ind_grid[s_1,s_1,s_1]], dim=-1),
    torch.stack([ind_grid[s_0,s_1,s_0], ind_grid[s_0,s_1,s_1], 
                 ind_grid[s_0,s_0,s_1], ind_grid[s_1,s_0,s_1]], dim=-1),
    torch.stack([ind_grid[s_0,s_0,s_0], ind_grid[s_0,s_1,s_0], 
                 ind_grid[s_0,s_0,s_1], ind_grid[s_1,s_0,s_1]], dim=-1),
], dim=-2)
indices = indices_grid.view(-1, 4)

In [ ]:
import matplotlib.pyplot as plt
def plot_tetrahedron_edges(vertices, c='blue'):
    """
    Plot the edges of a tetrahedron given its vertices.

    :param vertices: A 4x3 array of vertices.
    """
    # Define the edges that make up the tetrahedron
    edges = [
        (vertices[0], vertices[1]),
        (vertices[0], vertices[2]),
        (vertices[0], vertices[3]),
        (vertices[1], vertices[2]),
        (vertices[1], vertices[3]),
        (vertices[2], vertices[3])
    ]

    # Plot each edge of the tetrahedron
    for edge in edges:
        x_values = [edge[0][0], edge[1][0]]
        y_values = [edge[0][1], edge[1][1]]
        z_values = [edge[0][2], edge[1][2]]
        ax.plot(x_values, y_values, z_values, color=c)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the tetrahedron (edges only)
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 0]], 'blue')
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 1]], 'red')
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 2]], 'green')
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 3]], 'black')
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 4]])
plot_tetrahedron_edges(vertices[indices_grid[0,0,0, 5]])

plt.show()


In [ ]:
# import numpy as np
# tets = np.load(f'tets/128_tets.npz')
# vertices    = torch.tensor(tets['vertices'], dtype=torch.float32)
# indices  = torch.tensor(tets['indices'], dtype=torch.long)

In [ ]:
from dmtet import DMTet

# sdf of a circle
def sdf_circle(query, radius=0.3):
    center = torch.zeros(1, 3, device=query.device)+0.1
    dist = torch.norm(query-center, dim=-1)
    return dist - radius

def sdf_plane(query):

    return query[:, 0] - 0.1

dmtet = DMTet()

sdf = sdf_circle(vertices)
verts, faces, uvs, uv_idx = dmtet(vertices.cuda(), sdf.cuda(), indices.cuda())

In [ ]:
import trimesh
mesh = trimesh.Trimesh(vertices=verts.cpu(), faces=faces.cpu())
mesh.show()